In [1]:
#my approch
import os
import cv2
import numpy as np
import pandas as pd
from skimage.metrics import structural_similarity as ssim
from skimage.measure import shannon_entropy

import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.measure import shannon_entropy

def compute_metrics(original, enhanced):
    """
    Compute PSNR, SSIM, and Shannon Entropy between the original and enhanced images.
    
    Parameters:
        original (np.array): Original image (BGR, uint8).
        enhanced (np.array): Enhanced image (BGR, uint8).
    
    Returns:
        tuple: (PSNR, SSIM, Entropy)
    """
    # Ensure both images have the same size (resize enhanced if necessary)
    if original.shape != enhanced.shape:
        enhanced = cv2.resize(enhanced, (original.shape[1], original.shape[0]))
    
    # PSNR computation
    psnr_value = cv2.PSNR(original, enhanced)
    
    # Determine an appropriate win_size for SSIM based on the image dimensions.
    h, w = original.shape[:2]
    min_dim = min(h, w)
    
    # If the smallest dimension is less than 3 pixels, SSIM is unreliable.
    if min_dim < 3:
        ssim_value = np.nan
    else:
        # Choose a win_size that is odd and does not exceed min_dim.
        if min_dim < 7:
            win_size = min_dim if (min_dim % 2 == 1) else (min_dim - 1)
        else:
            win_size = 7  # default win_size for larger images
        
        # Compute SSIM using the computed win_size and specifying channel_axis=-1 for color images.
        try:
            ssim_value, _ = ssim(original, enhanced, full=True, channel_axis=-1, win_size=win_size)
        except ValueError as e:
            print(f"Error computing SSIM for image of size ({h}, {w}) with win_size={win_size}: {e}")
            ssim_value = np.nan

    # Compute Shannon entropy on the enhanced image.
    # Convert the enhanced image to grayscale.
    enhanced_gray = cv2.cvtColor(enhanced, cv2.COLOR_BGR2GRAY)
    entropy_value = shannon_entropy(enhanced_gray)
    
    return psnr_value, ssim_value, entropy_value



def process_folders(original_folder, enhanced_folder, output_csv=r"D:\mit-5k-subset\my_enhancement_metrics_parte.csv"):
    valid_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')
    results = []

    for filename in os.listdir(original_folder):
        if filename.lower().endswith(valid_extensions):
            orig_path = os.path.join(original_folder, filename)

            # Remove 'normal' prefix and add 'low' prefix
            basename = filename.replace('normal', 'low', 1)  # replace only the first occurrence
            enh_path = os.path.join(enhanced_folder, basename)

            if not os.path.exists(enh_path):
                print(f"Enhanced image for {filename} not found (expected {basename}). Skipping.")
                continue

            original_img = cv2.imread(orig_path)
            enhanced_img = cv2.imread(enh_path)

            if original_img is None or enhanced_img is None:
                print(f"Error reading {filename}. Skipping.")
                continue

            psnr_val, ssim_val, entropy_val = compute_metrics(original_img, enhanced_img)
            results.append({
                'Filename': filename,  # you can also use a common name like basename if you prefer
                'My PSNR': psnr_val,
                'My SSIM': ssim_val,
                'My Entropy': entropy_val
            })
            print(f"Processed {filename}: PSNR={psnr_val:.2f}, SSIM={ssim_val:.3f}, Entropy={entropy_val:.2f}")

    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)
    print(f"Results saved to {output_csv}")
    return df




    # Replace these with the paths to your folders.
original_folder = r"D:\mit-5k-subset\raw"
enhanced_folder = r"D:\mit-5k-subset\e_enhanced"
    
    # Process the folders and save metrics to a CSV.
metrics_df1 = process_folders(original_folder, enhanced_folder)


Processed a0034-LSYD4O2202.jpg: PSNR=11.27, SSIM=0.245, Entropy=7.94
Processed a0035-dgw_048.jpg: PSNR=11.68, SSIM=0.460, Entropy=7.46
Processed a0291-IMG_0115.jpg: PSNR=13.22, SSIM=0.262, Entropy=7.90
Processed a0436-IMG_2583.jpg: PSNR=14.36, SSIM=0.697, Entropy=7.90
Processed a0452-IMG_1646.jpg: PSNR=22.78, SSIM=0.736, Entropy=6.79
Processed a0463-jmac_DSC2316.jpg: PSNR=12.86, SSIM=0.476, Entropy=7.32
Processed a0631-NKIM_MG_6442.jpg: PSNR=17.95, SSIM=0.671, Entropy=7.21
Processed a0648-IMG_5085.jpg: PSNR=15.34, SSIM=0.608, Entropy=7.84
Processed a0712-_DSC8911.jpg: PSNR=17.16, SSIM=0.716, Entropy=7.58
Processed a0752-20061213_134314__MG_3708.jpg: PSNR=18.50, SSIM=0.531, Entropy=7.32
Processed a0767-jn_20070824_0165.jpg: PSNR=13.48, SSIM=0.292, Entropy=7.82
Processed a0770-050703_161554__I2E9266.jpg: PSNR=23.16, SSIM=0.657, Entropy=7.65
Processed a0775-kme_423.jpg: PSNR=11.40, SSIM=0.390, Entropy=7.88
Processed a0811-20051224_165428__MG_0953.jpg: PSNR=22.29, SSIM=0.884, Entropy=7.49


In [2]:
import matplotlib.pyplot as plt
# Compute Averages
average_values = metrics_df1.mean(numeric_only=True)
print("\nAverage Values of our approach:")
print(average_values)


Average Values of our approach:
My PSNR       14.787410
My SSIM        0.528294
My Entropy     7.690335
dtype: float64


In [3]:
#my approch
import os
import cv2
import numpy as np
import pandas as pd
from skimage.metrics import structural_similarity as ssim
from skimage.measure import shannon_entropy

import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.measure import shannon_entropy

def compute_metrics(original, enhanced):
    """
    Compute PSNR, SSIM, and Shannon Entropy between the original and enhanced images.
    
    Parameters:
        original (np.array): Original image (BGR, uint8).
        enhanced (np.array): Enhanced image (BGR, uint8).
    
    Returns:
        tuple: (PSNR, SSIM, Entropy)
    """
    # Ensure both images have the same size (resize enhanced if necessary)
    if original.shape != enhanced.shape:
        enhanced = cv2.resize(enhanced, (original.shape[1], original.shape[0]))
    
    # PSNR computation
    psnr_value = cv2.PSNR(original, enhanced)
    
    # Determine an appropriate win_size for SSIM based on the image dimensions.
    h, w = original.shape[:2]
    min_dim = min(h, w)
    
    # If the smallest dimension is less than 3 pixels, SSIM is unreliable.
    if min_dim < 3:
        ssim_value = np.nan
    else:
        # Choose a win_size that is odd and does not exceed min_dim.
        if min_dim < 7:
            win_size = min_dim if (min_dim % 2 == 1) else (min_dim - 1)
        else:
            win_size = 7  # default win_size for larger images
        
        # Compute SSIM using the computed win_size and specifying channel_axis=-1 for color images.
        try:
            ssim_value, _ = ssim(original, enhanced, full=True, channel_axis=-1, win_size=win_size)
        except ValueError as e:
            print(f"Error computing SSIM for image of size ({h}, {w}) with win_size={win_size}: {e}")
            ssim_value = np.nan

    # Compute Shannon entropy on the enhanced image.
    # Convert the enhanced image to grayscale.
    enhanced_gray = cv2.cvtColor(enhanced, cv2.COLOR_BGR2GRAY)
    entropy_value = shannon_entropy(enhanced_gray)
    
    return psnr_value, ssim_value, entropy_value



def process_folders(original_folder, enhanced_folder, output_csv=r"D:\mit-5k-subset\cidnet_enhancement_metrics_parte.csv"):
    valid_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')
    results = []

    for filename in os.listdir(original_folder):
        if filename.lower().endswith(valid_extensions):
            orig_path = os.path.join(original_folder, filename)

            # Remove 'normal' prefix and add 'low' prefix
            basename = filename.replace('normal', 'low', 1)  # replace only the first occurrence
            enh_path = os.path.join(enhanced_folder, basename)

            if not os.path.exists(enh_path):
                print(f"Enhanced image for {filename} not found (expected {basename}). Skipping.")
                continue

            original_img = cv2.imread(orig_path)
            enhanced_img = cv2.imread(enh_path)

            if original_img is None or enhanced_img is None:
                print(f"Error reading {filename}. Skipping.")
                continue

            psnr_val, ssim_val, entropy_val = compute_metrics(original_img, enhanced_img)
            results.append({
                'Filename': filename,  # you can also use a common name like basename if you prefer
                'CIDNet PSNR': psnr_val,
                'CIDNet SSIM': ssim_val,
                'CIDNet Entropy': entropy_val
            })
            print(f"Processed {filename}: PSNR={psnr_val:.2f}, SSIM={ssim_val:.3f}, Entropy={entropy_val:.2f}")

    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)
    print(f"Results saved to {output_csv}")
    return df




    # Replace these with the paths to your folders.
original_folder = r"D:\mit-5k-subset\raw"
enhanced_folder = r"D:\mit-5k-subset\e_enhanced_cidnet"
    
    # Process the folders and save metrics to a CSV.
metrics_df2 = process_folders(original_folder, enhanced_folder)


Processed a0034-LSYD4O2202.jpg: PSNR=7.82, SSIM=0.154, Entropy=7.81
Processed a0035-dgw_048.jpg: PSNR=13.29, SSIM=0.479, Entropy=7.50
Processed a0291-IMG_0115.jpg: PSNR=9.17, SSIM=0.215, Entropy=7.66
Processed a0436-IMG_2583.jpg: PSNR=6.14, SSIM=0.461, Entropy=6.49
Processed a0452-IMG_1646.jpg: PSNR=7.55, SSIM=0.553, Entropy=5.87
Processed a0463-jmac_DSC2316.jpg: PSNR=8.34, SSIM=0.427, Entropy=6.98
Processed a0631-NKIM_MG_6442.jpg: PSNR=8.95, SSIM=0.609, Entropy=5.50
Processed a0648-IMG_5085.jpg: PSNR=7.05, SSIM=0.446, Entropy=6.78
Processed a0712-_DSC8911.jpg: PSNR=5.77, SSIM=0.435, Entropy=6.83
Processed a0752-20061213_134314__MG_3708.jpg: PSNR=8.49, SSIM=0.437, Entropy=6.37
Processed a0767-jn_20070824_0165.jpg: PSNR=7.83, SSIM=0.203, Entropy=7.55
Processed a0770-050703_161554__I2E9266.jpg: PSNR=8.62, SSIM=0.596, Entropy=5.41
Processed a0775-kme_423.jpg: PSNR=4.41, SSIM=0.218, Entropy=6.47
Processed a0811-20051224_165428__MG_0953.jpg: PSNR=10.19, SSIM=0.811, Entropy=4.28
Processed a0

In [4]:
import matplotlib.pyplot as plt
# Compute Averages
average_values = metrics_df2.mean(numeric_only=True)
print("\nCIDnet Average Values:")
print(average_values)


CIDnet Average Values:
CIDNet PSNR       8.149436
CIDNet SSIM       0.425136
CIDNet Entropy    6.767673
dtype: float64


In [5]:
merged_df = pd.merge(metrics_df1, metrics_df2, on='Filename', how='inner') 



In [6]:
merged_df.head(8)

,Filename,My PSNR,My SSIM,My Entropy,CIDNet PSNR,CIDNet SSIM,CIDNet Entropy
0,a0034-LSYD4O2202.jpg,11.272995,0.244682,7.937058,7.817228,0.154251,7.806843
1,a0035-dgw_048.jpg,11.676046,0.460200,7.456725,13.286919,0.479173,7.497022
2,a0291-IMG_0115.jpg,13.222801,0.262181,7.901004,9.167654,0.214835,7.664426
3,a0436-IMG_2583.jpg,14.355215,0.697432,7.899609,6.137018,0.460679,6.485174
4,a0452-IMG_1646.jpg,22.776527,0.736363,6.792917,7.547662,0.553020,5.865075
5,a0463-jmac_DSC2316.jpg,12.857684,0.475697,7.317540,8.341126,0.427013,6.975190
6,a0631-NKIM_MG_6442.jpg,17.951300,0.671212,7.207311,8.954768,0.609108,5.500582
7,a0648-IMG_5085.jpg,15.335682,0.607879,7.835384,7.049795,0.445673,6.780024
